In [1]:
%matplotlib notebook
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import seaborn as sns
import colorlover as cl

py.offline.init_notebook_mode(connected=True)

In [2]:
fname0 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/ITU-MOM/ITU-MOM_GSexps_1980_2009.nc'
ds0 = xr.open_mfdataset(fname0)

fname1 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/MITB500/MIT_MITB500_1980_2009.nc'
ds1 = xr.open_mfdataset(fname1)

fname2 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/Brest/Brest_GSexps_1980_2009.nc'
ds2 = xr.open_mfdataset(fname2)

fname3 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/NorESM/NorESM_GS_experiments_1980_2009.nc'
ds3 = xr.open_mfdataset(fname3)

fname4 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/FESOM/FESOM_GS_exps_1980_2009.nc'
ds4 = xr.open_mfdataset(fname4)

In [3]:
dates0 = pd.date_range('1980-01', freq='M', periods=12*29)
ds0['time'] = dates0
ds1['time'] = dates0
ds2['time'] = dates0
ds3['time'] = dates0
ds4['time'] = dates0

In [6]:
""" Plot annual mean of volume fluxes anomaly"""
modelnames = ['ITU' , 'MITB500', 'Brest', 'NorESM', 'FESOM']
colors = ['blue', 'darkorange', 'red', 'green', 'cyan']
#colors2 = sns.color_palette("Paired", n_colors=10)
colors2 = cl.scales['10']['qual']['Paired'];  # make paired twice the amount of models

# names of the control, gsp and gsn variables [total]
ctln = ['ctlvice_barents', 'SIvolBK_c', 'refice_volume_BSKS', 'ice_volume_NorESM_BarentsKara_ctrl', 'ctlivol_BK']
gspn = ['gspvice_barents', 'SIvolBK_gsp', 'gspice_volume_BSKS', 'ice_volume_NorESM_BarentsKara_gsp', 'gspivol_BK']
gsnn = ['gsnvice_barents', 'SIvolBK_gsm', 'gsnice_volume_BSKS', 'ice_volume_NorESM_BarentsKara_gsn', 'gsnivol_BK']

#ctlhn = ['ctlbarents_heatinflow', 'barht_c[1]', 'BSOrefheat_in', 'inflow_ht_NorESM_BSO_ctrl', 'ctlheattransp_BSO_in']
#gsphn = ['gspbarents_heatinflow', 'barht_gsp[1]', 'BSOgspheat_in', 'inflow_ht_NorESM_BSO_gsp', 'gspheattransp_BSO_in']
#gsnhn = ['gsnbarents_heatinflow', 'barht_gsm[1]', 'BSOgsnheat_in', 'inflow_ht_NorESM_BSO_gsn', 'gsnheattransp_BSO_in']

ctlhn = ['ctlbarents_heattotal', 'barht_c[0]', 'BSOrefheat', 'ht_NorESM_BSO_ctrl', 'ctlheattransp_BSO']
gsphn = ['gspbarents_heattotal', 'barht_gsp[0]', 'BSOgspheat', 'ht_NorESM_BSO_gsp', 'gspheattransp_BSO']
gsnhn = ['gsnbarents_heattotal', 'barht_gsm[0]', 'BSOgsnheat', 'ht_NorESM_BSO_gsn', 'gsnheattransp_BSO']


coeffs = [1e-9, 1e-9, 1e-9, 1e-9, 1e-9]

layout = go.Layout(
    title='<b>Annual Heat vs Sea-ice  Extent Anomaly @BSO</b>',
    yaxis=dict(
        title='$\\text{Sea ice volume } [10^{-9} m^3]$',
        #range=[-14, 14]
    ),
    xaxis=dict(
        title='Heat Transport [TW]',
    )
)    


def update_plot(models):
    
    datap = []
    datan = []
    for ind,mdl in enumerate(models):
        dnm = 'ds'+np.str(ind)+'.'+ctln[ind]+'.groupby(\'time.year\').mean().year'
        time = np.copy(eval(dnm))
        dnm = 'ds'+np.str(ind)+'.'+ctln[ind]+'.groupby(\'time.year\').mean()'
        dnm2 = 'ds'+np.str(ind)+'.'+gspn[ind]+'.groupby(\'time.year\').mean()'
        dnm3 = 'ds'+np.str(ind)+'.'+gsnn[ind]+'.groupby(\'time.year\').mean()'
        val1 = np.copy(eval(dnm2)-eval(dnm))*coeffs[ind]
        val2 = np.copy(eval(dnm3)-eval(dnm))*coeffs[ind]
        dnm = 'ds'+np.str(ind)+'.'+ctlhn[ind]+'.groupby(\'time.year\').mean()'
        dnm2 = 'ds'+np.str(ind)+'.'+gsphn[ind]+'.groupby(\'time.year\').mean()'
        dnm3 = 'ds'+np.str(ind)+'.'+gsnhn[ind]+'.groupby(\'time.year\').mean()'
        val3 = np.copy(eval(dnm2)-eval(dnm))
        val4 = np.copy(eval(dnm3)-eval(dnm))
        trace1 = go.Scatter(            
            x = val3,
            y = val1,
            mode = 'lines',
            name = mdl+'-GSP',
            line = dict(
                width = 2.5,
                #color = colors[ind],
                color = colors2[2*ind],
                dash = 'dot',
                shape = 'smooth'
            )
        )
        datap.append(trace1)
        trace2 = go.Scatter(            
            x = val4,
            y = val2,
            mode = 'lines',
            name = mdl+'-GSM',
            line = dict(
                width = 2.5,
                #color = colors[ind],
                color = colors2[2*ind+1],
                dash = 'dot',
                shape = 'smooth'
            )
        )
        datan.append(trace2)


   
    data = datap + datan
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)
    
    
update_plot(modelnames)
#models = widgets.SelectMultiple(options=list(modelnames), value=modelnames, description='Models')    
#widgets.interactive(update_plot,models=models)

In [7]:
""" Plot seasonal mean of volume fluxes anomaly"""
modelnames = ['ITU' , 'MITB500', 'Brest', 'NorESM', 'FESOM']
colors = ['blue', 'darkorange', 'red', 'green', 'cyan']
colors2 = cl.scales['10']['qual']['Paired']; # make paired twice the amount of models

# names of the control, gsp and gsn variables [total]
ctln = ['ctlvice_barents', 'SIvolBK_c', 'refice_volume_BSKS', 'ice_volume_NorESM_BarentsKara_ctrl', 'ctlivol_BK']
gspn = ['gspvice_barents', 'SIvolBK_gsp', 'gspice_volume_BSKS', 'ice_volume_NorESM_BarentsKara_gsp', 'gspivol_BK']
gsnn = ['gsnvice_barents', 'SIvolBK_gsm', 'gsnice_volume_BSKS', 'ice_volume_NorESM_BarentsKara_gsn', 'gsnivol_BK']

#ctlhn = ['ctlbarents_heatinflow', 'barht_c[1]', 'BSOrefheat_in', 'inflow_ht_NorESM_BSO_ctrl', 'ctlheattransp_BSO_in']
#gsphn = ['gspbarents_heatinflow', 'barht_gsp[1]', 'BSOgspheat_in', 'inflow_ht_NorESM_BSO_gsp', 'gspheattransp_BSO_in']
#gsnhn = ['gsnbarents_heatinflow', 'barht_gsm[1]', 'BSOgsnheat_in', 'inflow_ht_NorESM_BSO_gsn', 'gsnheattransp_BSO_in']

ctlhn = ['ctlbarents_heattotal', 'barht_c[0]', 'BSOrefheat', 'ht_NorESM_BSO_ctrl', 'ctlheattransp_BSO']
gsphn = ['gspbarents_heattotal', 'barht_gsp[0]', 'BSOgspheat', 'ht_NorESM_BSO_gsp', 'gspheattransp_BSO']
gsnhn = ['gsnbarents_heattotal', 'barht_gsm[0]', 'BSOgsnheat', 'ht_NorESM_BSO_gsn', 'gsnheattransp_BSO']


coeffs = [1e-9, 1e-9, 1e-9, 1e-9, 1e-9]

layout = go.Layout(
    title='<b>Monthly Inflow Transport Anomaly @BSO</b>',
    yaxis=dict(
        title='$\\text{Sea ice volume } [10^{-9} m^3]$',
        #range=[-14, 14]
    ),
    xaxis=dict(
        title='Heat Transport [TW]',
    )
)    


def update_plot(models):
    
    datap = []
    datan = []
    for ind,mdl in enumerate(models):
        dnm = 'ds'+np.str(ind)+'.'+ctln[ind]+'.groupby(\'time.month\').mean().month'
        time = np.copy(eval(dnm))
        dnm = 'ds'+np.str(ind)+'.'+ctln[ind]+'.groupby(\'time.month\').mean()'
        dnm2 = 'ds'+np.str(ind)+'.'+gspn[ind]+'.groupby(\'time.month\').mean()'
        dnm3 = 'ds'+np.str(ind)+'.'+gsnn[ind]+'.groupby(\'time.month\').mean()'
        val1 = np.copy(eval(dnm2)-eval(dnm))*coeffs[ind]
        val2 = np.copy(eval(dnm3)-eval(dnm))*coeffs[ind]
        dnm = 'ds'+np.str(ind)+'.'+ctlhn[ind]+'.groupby(\'time.month\').mean()'
        dnm2 = 'ds'+np.str(ind)+'.'+gsphn[ind]+'.groupby(\'time.month\').mean()'
        dnm3 = 'ds'+np.str(ind)+'.'+gsnhn[ind]+'.groupby(\'time.month\').mean()'
        val3 = np.copy(eval(dnm2)-eval(dnm))
        val4 = np.copy(eval(dnm3)-eval(dnm))
        trace1 = go.Scatter(            
            x = val3,
            y = val1,
            mode = 'lines',
            name = mdl+'-GSP',        
            line = dict(
                width = 4,
                #color = colors[ind],
                color = colors2[2*ind],
                dash = 'dot',
                shape = 'smooth'
            )
        )
        datap.append(trace1)
        trace2 = go.Scatter(            
            x = val4,
            y = val2,
            mode = 'lines',
            name = mdl+'-GSM',            
            line = dict(
                width = 4,
                #color = colors[ind],
                color = colors2[2*ind+1],
                dash = 'dot',
                shape = 'smooth'
            )
        )
        datan.append(trace2)


   
    data = datap + datan
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)
    
update_plot(modelnames)    
#models = widgets.SelectMultiple(options=list(modelnames), value=modelnames, description='Models')    
#widgets.interactive(update_plot,models=models)

In [27]:
colors2

['rgb(166,206,227)',
 'rgb(31,120,180)',
 'rgb(178,223,138)',
 'rgb(51,160,44)',
 'rgb(251,154,153)',
 'rgb(227,26,28)',
 'rgb(253,191,111)',
 'rgb(255,127,0)',
 'rgb(202,178,214)',
 'rgb(106,61,154)']

In [ ]:
ds2.F